In [ ]:
# import packages
import mlflow
from mlflow.deployments import get_deploy_client
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_diabetes 
from sklearn.ensemble import RandomForestRegressor

In [ ]:
# Start local MLFlow server on CLI with SQLLite DB for model registry
# mlflow server \
#     --backend-store-uri sqlite:///mlflow.db \
#     --default-artifact-root ./artifacts \
#     --host 0.0.0.0

In [ ]:
# set mlflow registry and tracking URIs
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_registry_uri("http://localhost:5000")
 
mr_uri = mlflow.get_registry_uri()
print("Current registry uri: {}".format(mr_uri))
tracking_uri = mlflow.get_tracking_uri()
print("Current tracking uri: {}".format(tracking_uri))

In [ ]:
# load dataset
db = load_diabetes()
X = db.data
y = db.target
X_train, X_test, y_train, y_test = train_test_split(X, y)
 
# Enable autolog()
# mlflow.sklearn.autolog() requires mlflow 1.11.0 or above.
mlflow.sklearn.autolog()
 
# With autolog() enabled, all model parameters, a model score, and the fitted model are automatically logged.  
with mlflow.start_run() as run:  
  # Set the model parameters. 
  n_estimators = 100
  max_depth = 6
  max_features = 3
  # Create and train model.
  rf = RandomForestRegressor(n_estimators = n_estimators, max_depth = max_depth, max_features = max_features)
  rf.fit(X_train, y_train)
  # Use the model to make predictions on the test dataset.
  predictions = rf.predict(X_test)

In [ ]:
# log model
model_name = "mchrestkha-sklearn"
mlflow.sklearn.log_model(rf, model_name, registered_model_name=model_name)

In [ ]:
client = mlflow.tracking.MlflowClient()
client.list_registered_models()

In [ ]:
# deploy model to Vertex AI
client = mlflow.deployments.get_deploy_client("google_cloud")
 
model_version = 1
model_uri=f"models:/{model_name}/{model_version}"
print(model_uri)
 
deployment = client.create_deployment(
    name="mlflow_on_gcp",
    model_uri=model_uri)